# AI Cloud上のDriverless AI、OSS H2Oへのクライアントアクセス

ドキュメント：
- https://docs.h2o.ai/haic/latest/using-platform-token/
- https://docs.h2o.ai/enterprise-steam/latest-stable/docs/python-docs/index.html

In [2]:
import h2osteam

h2osteam.__version__

'1.8.12'

In [9]:
import getpass

In [10]:
# Steamへログイン（本セルを実行後、表示された入力ボックスへパスワード（トークン）を入力）
h2osteam.login(url="https://steam.cloud-internal.h2o.ai", 
               username="yuki.shimada@h2o.ai",
               password=getpass.getpass('Enter your API token here'),
               verify_ssl=False)

Enter your API token here········


#### トークンの取得
<img src="image/steam_token.png" width=600px>

## Driverless AIへの接続
- https://docs.h2o.ai/enterprise-steam/latest-stable/docs/python-docs/examples-k8s.html

In [16]:
h2osteam.clients.DriverlessClient().get_instances()

In [18]:
# 既存のインスタンスから選択
dai_instance = h2osteam.clients.DriverlessClient().get_instance(name="DAI_1_10_3_1")
dai_instance

In [19]:
dai_instance.status()

'stopped'

In [20]:
dai_instance.start()

Driverless AI instance is starting, please wait...
Driverless AI instance is running


In [21]:
dai_instance.status()

'running'

<img src="image/steam_dai.png" width=600px>

In [22]:
# Driverless AI Clientの取得
dai = dai_instance.connect()
dai

<class 'driverlessai._core.Client'> https://steam.cloud-internal.h2o.ai:443/proxy/driverless/687

Driverless AI Clientが取得できたので、後は通常のPython Client APIと同じ  
https://docs.h2o.ai/driverless-ai/pyclient/docs/html/index.html

In [24]:
# Driverless AIにアップされているデータのリスト
dai.datasets.list()

    | Type    | Key                                  | Name
----+---------+--------------------------------------+----------------------------
  0 | Dataset | 0482fc92-f349-11ec-a7da-dadce74b093e | movielens_train_sample.csv
  1 | Dataset | a7d90e1e-f348-11ec-a7da-dadce74b093e | movielens_test_sample.csv
  2 | Dataset | 3d42f85e-f342-11ec-a7da-dadce74b093e | superstore_orders_jp.csv
  3 | Dataset | 1f964384-f341-11ec-a7da-dadce74b093e | iris.csv

In [25]:
# Experimentのリスト
dai.experiments.list()

    | Type       | Key                                  | Name
----+------------+--------------------------------------+----------------
  0 | Experiment | 755bb8aa-f349-11ec-a7da-dadce74b093e | recommendation
  1 | Experiment | f1955eb0-f346-11ec-a7da-dadce74b093e | clustering-ohe
  2 | Experiment | e58da9bc-f344-11ec-a7da-dadce74b093e | clustering2
  3 | Experiment | 865b16a6-f343-11ec-a7da-dadce74b093e | clustering

In [52]:
# Driverless AIインスタンスの停止
dai_instance.stop()

Driverless AI instance is stopping, please wait...
Driverless AI instance is stopped


## OSS H2Oへの接続
- https://docs.h2o.ai/enterprise-steam/latest-stable/docs/python-docs/examples-k8s.html

In [55]:
h2osteam.clients.H2oKubernetesClient().get_clusters()

[]

In [56]:
# H2Oクラスターの作成
h2o_cluster = h2osteam.clients.H2oKubernetesClient().launch_cluster(name="h2o-cluster", version="3.36.1.2")
h2o_cluster

<img src="image/steam_h2o.png" width=600px>

In [57]:
h2o_cluster.status

'created'

In [58]:
# AI Cloud上のH2Oに接続
h2o_cluster.connect()

Connecting to H2O server at https://steam.cloud-internal.h2o.ai:443/proxy/h2o-k8s/271 .... successful.


H2O_cluster_uptime:,1 min 42 secs
H2O_cluster_timezone:,Etc/GMT
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.2
H2O_cluster_version_age:,1 month and 2 days
H2O_cluster_name:,h2o-cluster
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.185 Gb
H2O_cluster_total_cores:,1
H2O_cluster_allowed_cores:,1
H2O_cluster_status:,"locked, healthy"


In [68]:
import h2o

h2o.__version__

'3.36.0.4'

H2Oクラスターに接続されたので、あとは通常のH2Oプログラムと同じ
- https://docs.h2o.ai/h2o/latest-stable/h2o-docs/index.html
- https://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/index.html

In [60]:
frame = h2o.upload_file("data/TitanicData2.csv", destination_frame='titanic')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


#### アップロードされたデータをFlowで確認
<img src="image/flow.png" width=600px>

In [64]:
frame.head(5)

Passenger_Id,pclass,survived,name_with_salutions,name_without_salutions,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,home_dest
1,3rd,0,"Andersson, Mr. Anders Johan","Andersson, Anders Johan",male,39,1,5,347082,31.275,,Southampton,nan,"Sweden Winnipeg, MN"
2,3rd,1,"McGowan, Miss. Anna ""Annie""","McGowan, Anna ""Annie""",female,15,0,0,330923,8.0292,,Queenstown,nan,
3,2nd,1,"Caldwell, Mr. Albert Francis","Caldwell, Albert Francis",male,26,1,1,248738,29,,Southampton,13,"Bangkok, Thailand / Roseville, IL"
4,3rd,0,"Augustsson, Mr. Albert","Augustsson, Albert",male,23,0,0,347468,7.8542,,Southampton,nan,"Krakoryd, Sweden Bloomington, IL"
5,3rd,1,"Lindqvist, Mr. Eino William","Lindqvist, Eino William",male,20,1,0,nan,7.925,,Southampton,15,


In [65]:
# H2Oクラスターの削除
h2o_cluster.terminate()